# Probe Designer


## Environment


In [1]:
# basci env
import os
import sys
import pandas as pd
import time
import json

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# get gene data from ncbi
from Bio import Entrez

# blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append("../lib")

# dir
project_name = "2023.11.4_Embryo_100"
os.makedirs(f"../dataset/{project_name}", exist_ok=True)
os.chdir(f"../dataset/{project_name}")

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)
tmp = "./results/" + formatted_time + "/tmp/"
output = "./results/" + formatted_time + "/"
pre_binding_dir = tmp + "pre_binding/"
os.makedirs(tmp, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total_pre_binding.fasta"

# tmp file
gene_name_list_file = "1_gene_name_list.txt"
gene_id_name_file = "2_id_list.txt"
gene_seq_in_file = "3_gene_seq_in_file.gb"
pre_binding_num_file = "4_pre_binding_num.json"
blast_results_file = "5_blast_results.xml"

## Get genbank file of each gene from ncbi dataset

https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch


In [2]:
# Get gene id and other information from ncbi dataset(api)
## Generate gene_search_list from gene_name_list
organism_of_interest = "Homo sapiens"
n_type_of_interest = "mRNA"
with open(tmp + gene_name_list_file) as f:
    gene_name_list = f.read().splitlines()
gene_search_list = [
    ", ".join([name, organism_of_interest, n_type_of_interest])
    for name in gene_name_list
]

In [161]:
## Get gene id list using Entrez.esearch
id_list = []
for gene_search in gene_search_list:
    Entrez.email = "1418767067@qq.com"
    handle = Entrez.esearch(db="nuccore", term=gene_search)
    record = Entrez.read(handle)
    handle.close()
    id_list += record["IdList"][:1]  # set number of search results to read
with open(tmp + gene_id_name_file, "w") as f:
    f.write("\n".join(id_list))

In [3]:
## Read id_list from existing file
with open(tmp + gene_id_name_file, "r") as f:
    id_list = f.read().split("\n")

In [8]:
# Get the genbank file of each gene by id list
fetch_per_round = 3
round = -(-len(id_list) // fetch_per_round)

# initialization of gb file
with open(tmp + gene_seq_in_file, "w") as f:
    f.write("")

for i in range(round):
    id_list_per_round = id_list[i * fetch_per_round : (i + 1) * fetch_per_round]
    Entrez.email = "1418767067@qq.com"
    handle = Entrez.efetch(
        db="nuccore",
        strand=1,  # plus if strand=1
        id=id_list_per_round,
        rettype="gbwithparts",
        retmode="text",
    )
    seq_record = handle.read()
    handle.close()
    print(i + 1, "{:.1f} %".format((i + 1) / round * 100))
    with open(tmp + gene_seq_in_file, "a") as f:
        f.write(seq_record)

1 12.5 %
2 25.0 %
3 37.5 %
4 50.0 %
5 62.5 %
6 75.0 %
7 87.5 %
8 100.0 %


## Binding site Searcher


In [4]:
sys.path.append("../lib")

In [5]:
from search_method import step_by_step, find_max_min_difference_fixed_length_subsequence, gb_extract

# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_in = tmp + gene_seq_in_file
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(file_out_dir + total_pre_binding_file_name, "w") as handle:
    handle.write("")

for record in SeqIO.parse(tmp + gene_seq_in_file, "genbank"):
    id, gene_name, mol_type, organism, seq = gb_extract(record, CDS=True)

    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=gene_name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
    )
    
    record_list = []
    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    file_out = file_out_dir + gene_name + pre_binding_file_suffix
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(tmp + pre_binding_num_file, "w") as f:
    json.dump(pre_binding_num, f)

position_searching_Arhgap36:  48%|████▊     | 1143/2401 [00:00<00:00, 11415.94it/s]

position_searching_Arhgap36: 100%|██████████| 2401/2401 [00:00<00:00, 12506.72it/s]


Gene Arhgap36: 	condition too harsh, loose to get better results
[27, 47, 153, 261, 361, 442, 518, 562, 628, 647, 825, 847, 890, 909, 928, 947, 966, 1098, 1117, 1136, 1155, 1232, 1254, 1275, 1336, 1484, 1505, 1570, 1604, 1624, 1643, 1670, 1690, 1926, 1950, 1969, 1988, 2007, 2046, 2065, 2084, 2201, 2220, 2239, 2258, 2278, 2330, 2349, 2373, 2393]


position_searching_Cacna2d3: 100%|██████████| 2928/2928 [00:00<00:00, 13930.72it/s]


Gene Cacna2d3: 	condition too harsh, loose to get better results
[5, 41, 63, 142, 164, 227, 250, 272, 316, 338, 360, 429, 460, 489, 683, 705, 785, 807, 846, 901, 971, 993, 1015, 1037, 1075, 1101, 1181, 1214, 1236, 1258, 1291, 1319, 1603, 1702, 1789, 1816, 1838, 1861, 1898, 1950, 2023, 2105, 2127, 2149, 2175, 2197, 2263, 2285, 2380, 2402, 2479, 2690]


...

position_searching_Ptprz1: 100%|██████████| 4368/4368 [00:00<00:00, 18279.46it/s]


Gene Ptprz1: 	condition too harsh, loose to get better results
[219, 287, 308, 329, 453, 611, 872, 962, 995, 1019, 1040, 1127, 1179, 1200, 1388, 1409, 1435, 1456, 1477, 1498, 1521, 1543, 1779, 1800, 1948, 1969, 1990, 2277, 2371, 2432, 2453, 2508, 2540, 2598, 2619, 2640, 2717, 2846, 2867, 3038, 3059, 3200, 3305, 3413, 3574, 3639, 3660, 3681, 3920, 4177]


position_searching_Syt4: 100%|██████████| 3081/3081 [00:00<00:00, 28609.49it/s]


Gene Syt4: 	condition too harsh, loose to get better results
[148, 301, 690, 1191, 1196, 1685, 1694, 1699, 1968, 1974, 1991, 1996, 2003, 2252, 2311, 2317, 2322, 2328, 2380, 2546, 2612, 2679, 2684, 2689, 2694, 2699, 2704, 2709, 2714, 2792, 2798, 2803, 2808, 2814, 2819, 2827, 2834, 2839, 2859, 2887, 2893, 2907, 2912, 2917, 2924, 2929, 2934, 2939, 2944, 2952, 2963, 2968, 2983, 2988, 2993]


position_searching_Wwp2: 100%|██████████| 3964/3964 [00:00<00:00, 10014.90it/s]


## Blast and extract blast results

NCBIXML: https://homolog.us/Biopython/Bio.Blast.NCBIXML.html#read/0

BlastRecord: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html

XMLReader: https://codebeautify.org/xmlviewer#


In [ ]:
with open(file_out_dir + total_pre_binding_file_name, "r") as f:
    fasta_string = f.read()
txid = [2697049]  # organism

# Submit BLAST search and get handle object
handle = NCBIWWW.qblast(
    program="blastn",
    megablast="yes",
    database="refseq_rna",
    sequence=fasta_string,
    url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
    format_object="Alignment",
    format_type="Xml",
)

# read handle object and save to a file
with open(tmp + blast_results_file, "w") as f:
    f.write(handle.read())

In [6]:
# Extract interested information from blast_results
align_num = []

# read the id/plus-minus part/align_num
with open(tmp + blast_results_file, "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [7]:
FOI["wanted"] = [True] * len(FOI)

In [8]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_name_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "mRNA":
            FOI.loc[i, "wanted"] = False

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(tmp + "probes_sieve.xlsx")

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(output + "probes_wanted.xlsx")